<a href="https://colab.research.google.com/github/3dsf/ArtLineFFMPEG/blob/main/ArtLineFFMPEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ArtLine Video Conversion
implementation of 
https://github.com/vijishmadhavan/ArtLine



# Setup


## Install Libs

In [1]:
!apt install ffmpeg
!pip install youtube-dl fastai==1.0.61 ffmpeg-python PyDrive

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
     |████████████████████████████████| 1.9MB 24.1MB/s 


In [2]:
import os
os.makedirs("input", exist_ok=True)
os.makedirs("output", exist_ok=True)

## Download Infernce Code

In [3]:
!wget -O run.py https://gist.githubusercontent.com/3dsf/5827842bd6dc4fe7ada5dda868eb4599/raw/6ffc0b5010e30a9e52af684f752adb2142718961/run.py

--2020-12-29 20:00:04--  https://gist.githubusercontent.com/3dsf/5827842bd6dc4fe7ada5dda868eb4599/raw/6ffc0b5010e30a9e52af684f752adb2142718961/run.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3931 (3.8K) [text/plain]
Saving to: ‘run.py’

run.py              100%[===================>]   3.84K  --.-KB/s    in 0s      

2020-12-29 20:00:04 (79.2 MB/s) - ‘run.py’ saved [3931/3931]



# ---> Select your video to process

In [4]:
youtubeVideo = "https://www.youtube.com/watch?v=pBkHHoOIIn8" #@param {type:"string"}
!rm input.mp4
!time(youtube-dl -f '[ext=mp4]' --output "input.%(ext)s"  $youtubeVideo)

rm: cannot remove 'input.mp4': No such file or directory
[youtube] pBkHHoOIIn8: Downloading webpage
[youtube] pBkHHoOIIn8: Downloading js player 5dd3f3b2
[youtube] pBkHHoOIIn8: Downloading js player 5dd3f3b2
[download] Destination: input.mp4
[download] 100% of 9.43MiB in 00:00

real	0m1.770s
user	0m1.115s
sys	0m0.083s


# Decode Video

In [5]:
#Audio
!rm audio.mp3
!time(ffmpeg -i input.mp4  audio.mp3 -y) 

rm: cannot remove 'audio.mp3': No such file or directory
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enab

In [6]:
#Visual
!rm input/*.bmp output/*.bmp
!time(ffmpeg -i input.mp4 input/frame.%05d.bmp)

rm: cannot remove 'input/*.bmp': No such file or directory
rm: cannot remove 'output/*.bmp': No such file or directory
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorb

# ---> Select the Model to Use
If you **didn't connect** to your **drive** and copy the models, **automatic fallback** is provided. 

In [7]:
artLineModel = "ArtLine_500.pkl" #@param ["ArtLine_500.pkl", "ArtLine_650.pkl", "ArtLine_1024.pkl"]
pathToModel = os.path.join("/content/drive/",artLineModel)

downloadModel = {
    "ArtLine_500.pkl": "https://www.dropbox.com/s/p9lynpwygjmeed2/ArtLine_500.pkl",
    "ArtLine_650.pkl": "https://www.dropbox.com/s/starqc9qd2e1lg1/ArtLine_650.pkl",
    "ArtLine_1024.pkl": "https://www.dropbox.com/s/rq90q9lr9arwdp8/ArtLine_1024%20%281%29.pkl"
}
import os.path as path
import subprocess
if path.isfile(pathToModel) == False : 
    if path.isfile(artLineModel) == False :
        import subprocess
        print("Downloading")
        subprocess.call("wget -O " + artLineModel + " " + downloadModel[artLineModel] , shell=True)
        pathToModel = artLineModel
    else :
        print("Found Local Version")
        pathToModel = artLineModel

Downloading


# Run Model

In [ ]:
!time(python run.py $pathToModel)


MODEL FOUND : ArtLine_500.pkl
INFERENCE DEVICE : cuda
MODEL LOADED IN : 4.6921646290000005
not sure the time tracking is accurate in colab
input/frame.03605.bmp (1/4080) time : 0.15987659500000007
input/frame.00961.bmp (2/4080) time : 0.12572727200000067
input/frame.01720.bmp (3/4080) time : 0.12214396300000008
input/frame.01052.bmp (4/4080) time : 0.09848375700000034
input/frame.00264.bmp (5/4080) time : 0.09788655000000013
input/frame.00269.bmp (6/4080) time : 0.09877647800000044
input/frame.01106.bmp (7/4080) time : 0.09753836600000021
input/frame.00796.bmp (8/4080) time : 0.09817080299999947
input/frame.00129.bmp (9/4080) time : 0.09945293700000057
input/frame.00347.bmp (10/4080) time : 0.09920101999999975
input/frame.03253.bmp (11/4080) time : 0.09815412700000081
input/frame.00418.bmp (12/4080) time : 0.09807518100000046
input/frame.01117.bmp (13/4080) time : 0.0982295310000012
input/frame.03050.bmp (14/4080) time : 0.09869219900000026
input/frame.02043.bmp (15/4080) time : 0.101

# ---> Select the Output Name and **Encode**


In [ ]:
output_name = "output.webm" #@param {type:"string"}
!time(ffmpeg -framerate `ffmpeg -i input.mp4 2>&1 | sed -n "s/.*, \(.*\) fp.*/\1/p"` -thread_queue_size 8192 -i output/frame.%05d.bmp   -i audio.mp3  $output_name -y)

# Download result

In [ ]:
from google.colab import files

files.download(output_name)